**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [3]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, adam
from keras.layers import Flatten, Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [5]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [6]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> The purpose of this function is, for a given state, to choose the next action. It is supposed to be used for learning. This is where $\epsilon$ is essential : it controls the trade-off between exploration (try new actions to receive feedback and learn it) and exploitation (follow the learned policy to maximize the reward). When small the emphasis is on exploitation, when large it is on exploration.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [49]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [110]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=200 # set small when debugging
epochs_train_explore = 150
epochs_test=50 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

> - ```position``` encodes the forbidden, authorized and visited positions : -1 if forbidden, 0 if authorized but still not visited and 1 if authorized and visited.
> - ```board``` encodes the cheese ($0.5$) or the poison ($-1$) in the same positions, or a neutral cell ($0$)

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [51]:

class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        # Choose uniformly among the random actions
        action = np.random.randint(0, self.n_action, size=1)[0]
        return action

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [52]:

def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [53]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 7.0/17.0. Average score (-10.0)
Win/lose count 7.0/8.0. Average score (-5.5)
Win/lose count 11.0/11.0. Average score (-3.6666666666666665)
Win/lose count 10.0/16.0. Average score (-4.25)
Win/lose count 10.0/15.0. Average score (-4.4)
Win/lose count 7.5/15.0. Average score (-4.916666666666667)
Win/lose count 5.5/15.0. Average score (-5.571428571428571)
Win/lose count 9.0/14.0. Average score (-5.5)
Win/lose count 12.0/13.0. Average score (-5.0)
Win/lose count 8.0/13.0. Average score (-5.0)
Win/lose count 9.0/19.0. Average score (-5.454545454545454)
Win/lose count 11.0/15.0. Average score (-5.333333333333333)
Win/lose count 11.0/15.0. Average score (-5.230769230769231)
Win/lose count 12.0/16.0. Average score (-5.142857142857143)
Win/lose count 5.5/9.0. Average score (-5.033333333333333)
Win/lose count 9.5/17.0. Average score (-5.1875)
Win/lose count 6.5/21.0. Average score (-5.735294117647059)
Win/lose count 9.0/15.0. Average score (-5.75)
Win/lose count 12.5/17.0. Average 

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




> We note that : 
\begin{align}
Q^\pi(s,a) & = E_{p^{\pi}}[\sum_{t \geq 0}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
& =r(s,a) + E_{p^{\pi}}[\sum_{t \geq 1}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
& =r(s,a) + \gamma \sum_{s',a'} p_{\pi}(s',a'|s,a)E_{p^{\pi}}[ \sum_{t \geq 1}\gamma^{t-1}r(s_{t},a_{t})|s_{1}=s',a_{1}=a'] \\
& =r(s,a) + \gamma \sum_{s',a'} p_{\pi}(s',a'|s,a)E_{p^{\pi}}[ \sum_{t \geq 0}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a'] \\
& =E_{(s',a')\sim p_{\pi}(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align}

>Then:
\begin{align}
Q^*(s,a) & = max_{\pi} Q^{\pi}(s,a) \\
&= max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi'}(s',a')] \\
&= r(s,a) + \gamma max_{\pi=(\pi_0,\pi')} \sum_{s',a'} p_{\pi_0}(s',a'|s,a)Q^{\pi'}(s',a') \\
&= r(s,a)+\gamma\max_{\pi_0}\sum_{s',a'} p(s'|s,a)\pi_0(a'|s')max_{\pi'}Q^{\pi'}(s',a') \\
&= r(s,a)+\gamma\sum_{s'} p(s'|s,a)\sum_{a'}\max_{\pi_0}\pi_0(a'|s')Q^*(s',a') \\
&= r(s,a)+\gamma\sum_{s'} p(s'|s,a)\max_{a'}Q^*(s',a') \\
&= E_{s' \sim p(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]
\end{align}

>Thus, we note that:
$0 = E_{s' \sim p(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')] - Q^*(s,a) = E_{s' \sim p(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a') - Q^*(s,a)]$
which means that, "on average", $r(s,a)+\gamma\max_{a'}Q^{*}(s',a') = Q^*(s,a)$. So if we want to measure to which extent this equality is true, we may want to check the variance, hence the objective function.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [54]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, move):
        
        # Add moves to the memory
        self.memory.append(move)
            
        # Crop to maximum memory size
        if len(self.memory) > self.max_memory:
            self.memory = self.memory[1:]
            
    def random_access(self):
        
        # Choose random indice
        random_ind = np.random.randint(0, len(self.memory))
        
        # Select and return corresponding move
        move = self.memory[random_ind]
        return move
                                       
        

***
The pipeline we will use for training is given below:

In [55]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [71]:

class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size
    

    def learned_act(self, s):
        pass

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        input_next_states = np.zeros((self.batch_size, 5,5,self.n_state))
        
        
        samples = []
        
        # Retrive memory samples and store current and next states
        for i in range(self.batch_size):
            move = self.memory.random_access()
            samples.append(move)
            state, next_state, action, reward, game_over = move
            input_states[i] = state
            input_next_states[i] = next_state
            
        # Compute/update current-state and next-state action-value function
        target_q = self.model.predict(input_states)
        target_q_next = self.model.predict(input_next_states)
        
        # Update ground truth target action-value function
        for i in range(self.batch_size):
            state, next_state, action, reward, game_over = samples[i]
            if game_over_:
                target_q[i,action] += reward
            else:
                target_q[i,action] += reward + self.discount*np.max(target_q_next[i])
                
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(20, activation='relu'))
        model.add(Dense(4))
        
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [72]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train190.mp4'))

Epoch 000/200 | Loss 0.0063 | Win/lose count 1.5/4.0 (-2.5)
Epoch 001/200 | Loss 0.0062 | Win/lose count 2.0/3.0 (-1.0)
Epoch 002/200 | Loss 0.0100 | Win/lose count 1.0/1.0 (0.0)
Epoch 003/200 | Loss 0.0028 | Win/lose count 1.5/3.0 (-1.5)
Epoch 004/200 | Loss 0.0016 | Win/lose count 2.0/1.0 (1.0)
Epoch 005/200 | Loss 0.0025 | Win/lose count 2.0/4.0 (-2.0)
Epoch 006/200 | Loss 0.0106 | Win/lose count 0/2.0 (-2.0)
Epoch 007/200 | Loss 0.0020 | Win/lose count 1.0/5.0 (-4.0)
Epoch 008/200 | Loss 0.0092 | Win/lose count 2.0/2.0 (0.0)
Epoch 009/200 | Loss 0.0021 | Win/lose count 4.0/1.0 (3.0)
Epoch 010/200 | Loss 0.0013 | Win/lose count 2.5/1.0 (1.5)
Epoch 011/200 | Loss 0.0005 | Win/lose count 1.0/2.0 (-1.0)
Epoch 012/200 | Loss 0.0101 | Win/lose count 3.0/1.0 (2.0)
Epoch 013/200 | Loss 0.0088 | Win/lose count 2.5/1.0 (1.5)
Epoch 014/200 | Loss 0.0006 | Win/lose count 3.0/2.0 (1.0)
Epoch 015/200 | Loss 0.0086 | Win/lose count 3.0/1.0 (2.0)
Epoch 016/200 | Loss 0.0009 | Win/lose count 2.0/3.

Epoch 139/200 | Loss 0.0001 | Win/lose count 3.5/0 (3.5)
Epoch 140/200 | Loss 0.0001 | Win/lose count 1.5/4.0 (-2.5)
Epoch 141/200 | Loss 0.0000 | Win/lose count 2.0/2.0 (0.0)
Epoch 142/200 | Loss 0.0000 | Win/lose count 3.0/0 (3.0)
Epoch 143/200 | Loss 0.0001 | Win/lose count 1.0/3.0 (-2.0)
Epoch 144/200 | Loss 0.0002 | Win/lose count 2.5/3.0 (-0.5)
Epoch 145/200 | Loss 0.0001 | Win/lose count 2.5/4.0 (-1.5)
Epoch 146/200 | Loss 0.0157 | Win/lose count 1.5/2.0 (-0.5)
Epoch 147/200 | Loss 0.0079 | Win/lose count 2.0/1.0 (1.0)
Epoch 148/200 | Loss 0.0000 | Win/lose count 1.5/0 (1.5)
Epoch 149/200 | Loss 0.0000 | Win/lose count 0.5/3.0 (-2.5)
Epoch 150/200 | Loss 0.0079 | Win/lose count 2.0/3.0 (-1.0)
Epoch 151/200 | Loss 0.0157 | Win/lose count 1.5/3.0 (-1.5)
Epoch 152/200 | Loss 0.0001 | Win/lose count 1.0/0 (1.0)
Epoch 153/200 | Loss 0.0078 | Win/lose count 2.0/2.0 (0.0)
Epoch 154/200 | Loss 0.0078 | Win/lose count 2.0/3.0 (-1.0)
Epoch 155/200 | Loss 0.0000 | Win/lose count 1.0/3.0 (-

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [92]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        # Convolutional layers
        model.add(Conv2D(32, (3,3), activation='relu', input_shape=(5, 5, self.n_state), padding='same'))
        model.add(Conv2D(16, (3,3), activation='relu'))
        model.add(Conv2D(8, (3,3), activation='relu'))
        # Final FC layer
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [93]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/200 | Loss 0.0000 | Win/lose count 3.0/3.0 (0.0)
Epoch 001/200 | Loss 0.0000 | Win/lose count 1.0/0 (1.0)
Epoch 002/200 | Loss 0.0000 | Win/lose count 0.5/2.0 (-1.5)
Epoch 003/200 | Loss 0.0000 | Win/lose count 2.5/3.0 (-0.5)
Epoch 004/200 | Loss 0.0000 | Win/lose count 2.0/3.0 (-1.0)
Epoch 005/200 | Loss 0.0078 | Win/lose count 3.0/1.0 (2.0)
Epoch 006/200 | Loss 0.0000 | Win/lose count 1.0/1.0 (0.0)
Epoch 007/200 | Loss 0.0078 | Win/lose count 2.5/0 (2.5)
Epoch 008/200 | Loss 0.0000 | Win/lose count 1.5/2.0 (-0.5)
Epoch 009/200 | Loss 0.0000 | Win/lose count 1.0/2.0 (-1.0)
Epoch 010/200 | Loss 0.0078 | Win/lose count 2.5/0 (2.5)
Epoch 011/200 | Loss 0.0000 | Win/lose count 2.0/4.0 (-2.0)
Epoch 012/200 | Loss 0.0000 | Win/lose count 1.5/3.0 (-1.5)
Epoch 013/200 | Loss 0.0000 | Win/lose count 4.0/0 (4.0)
Epoch 014/200 | Loss 0.0078 | Win/lose count 0/5.0 (-5.0)
Epoch 015/200 | Loss 0.0000 | Win/lose count 3.0/6.0 (-3.0)
Epoch 016/200 | Loss 0.0000 | Win/lose count 2.5/4.0 (-1.

Epoch 139/200 | Loss 0.0000 | Win/lose count 4.0/1.0 (3.0)
Epoch 140/200 | Loss 0.0000 | Win/lose count 3.0/2.0 (1.0)
Epoch 141/200 | Loss 0.0000 | Win/lose count 1.5/1.0 (0.5)
Epoch 142/200 | Loss 0.0000 | Win/lose count 1.0/1.0 (0.0)
Epoch 143/200 | Loss 0.0000 | Win/lose count 1.5/1.0 (0.5)
Epoch 144/200 | Loss 0.0000 | Win/lose count 1.0/2.0 (-1.0)
Epoch 145/200 | Loss 0.0000 | Win/lose count 2.5/2.0 (0.5)
Epoch 146/200 | Loss 0.0078 | Win/lose count 0/6.0 (-6.0)
Epoch 147/200 | Loss 0.0000 | Win/lose count 1.0/2.0 (-1.0)
Epoch 148/200 | Loss 0.0000 | Win/lose count 3.0/2.0 (1.0)
Epoch 149/200 | Loss 0.0078 | Win/lose count 2.5/1.0 (1.5)
Epoch 150/200 | Loss 0.0000 | Win/lose count 2.5/5.0 (-2.5)
Epoch 151/200 | Loss 0.0078 | Win/lose count 2.0/3.0 (-1.0)
Epoch 152/200 | Loss 0.0000 | Win/lose count 3.0/5.0 (-2.0)
Epoch 153/200 | Loss 0.0078 | Win/lose count 2.0/0 (2.0)
Epoch 154/200 | Loss 0.0156 | Win/lose count 3.0/4.0 (-1.0)
Epoch 155/200 | Loss 0.0156 | Win/lose count 2.0/3.0 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [94]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 2.5/1.0. Average score (1.5)
Win/lose count 2.0/1.0. Average score (1.25)
Win/lose count 2.0/2.0. Average score (0.8333333333333334)
Win/lose count 2.0/3.0. Average score (0.375)
Win/lose count 0.5/3.0. Average score (-0.2)
Win/lose count 2.0/5.0. Average score (-0.6666666666666666)
Win/lose count 1.0/3.0. Average score (-0.8571428571428571)
Win/lose count 1.0/1.0. Average score (-0.75)
Win/lose count 1.0/1.0. Average score (-0.6666666666666666)
Win/lose count 3.0/3.0. Average score (-0.6)
Win/lose count 2.0/3.0. Average score (-0.6363636363636364)
Win/lose count 2.0/1.0. Average score (-0.5)
Win/lose count 2.0/0. Average score (-0.3076923076923077)
Win/lose count 1.0/6.0. Average score (-0.6428571428571429)
Win/lose count 0.5/3.0. Average score (-0.7666666666666667)
Win/lose count 1.5/1.0. Average score (-0.6875)
Win/lose count 0.5/4.0. Average score (-0.8529411764705882)
Win/lose count 2.5/3.0. Average score (-0.8333333333333334)
Win/lose count 2.5/2.0.

In [95]:
HTML(display_videos('cnn_test10.mp4'))

In [96]:
HTML(display_videos('fc_test10.mp4'))

In [101]:
env = Environment(grid_size=size, max_time=T,temperature=0.1)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 0/3.0. Average score (-3.0)
Win/lose count 0.5/0. Average score (-1.25)
Win/lose count 0.5/0. Average score (-0.6666666666666666)
Win/lose count 1.0/1.0. Average score (-0.5)
Win/lose count 1.5/0. Average score (-0.1)
Win/lose count 1.0/1.0. Average score (-0.08333333333333333)
Win/lose count 0.5/1.0. Average score (-0.14285714285714285)
Win/lose count 1.0/2.0. Average score (-0.25)
Win/lose count 0/2.0. Average score (-0.4444444444444444)
Win/lose count 0/3.0. Average score (-0.7)
Win/lose count 0.5/0. Average score (-0.5909090909090909)
Win/lose count 0/0. Average score (-0.5416666666666666)
Win/lose count 0.5/2.0. Average score (-0.6153846153846154)
Win/lose count 0/5.0. Average score (-0.9285714285714286)
Win/lose count 0.5/0. Average score (-0.8333333333333334)
Win/lose count 0/1.0. Average score (-0.84375)
Win/lose count 0.5/0. Average score (-0.7647058823529411)
Win/lose count 0.5/0. Average score (-0.6944444444444444)
Win/lose count 1.0/0. Average

In [102]:
HTML(display_videos('cnn_test10.mp4'))

In [103]:
HTML(display_videos('fc_test10.mp4'))

In [107]:
env = Environment(grid_size=size, max_time=T,temperature=0.5)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 3.0/3.0. Average score (0.0)
Win/lose count 2.0/2.0. Average score (0.0)
Win/lose count 4.5/2.0. Average score (0.8333333333333334)
Win/lose count 2.0/4.0. Average score (0.125)
Win/lose count 4.0/4.0. Average score (0.1)
Win/lose count 3.0/3.0. Average score (0.08333333333333333)
Win/lose count 3.0/2.0. Average score (0.21428571428571427)
Win/lose count 4.5/3.0. Average score (0.375)
Win/lose count 3.5/2.0. Average score (0.5)
Win/lose count 3.5/2.0. Average score (0.6)
Win/lose count 1.5/1.0. Average score (0.5909090909090909)
Win/lose count 5.5/2.0. Average score (0.8333333333333334)
Win/lose count 2.5/3.0. Average score (0.7307692307692307)
Win/lose count 4.5/5.0. Average score (0.6428571428571429)
Win/lose count 2.5/2.0. Average score (0.6333333333333333)
Win/lose count 4.0/2.0. Average score (0.71875)
Win/lose count 3.0/2.0. Average score (0.7352941176470589)
Win/lose count 3.0/2.0. Average score (0.75)
Win/lose count 1.0/1.0. Average score (0.71052

In [105]:
HTML(display_videos('cnn_test10.mp4'))

In [106]:
HTML(display_videos('fc_test10.mp4'))

We note that the CNN agent usually outperforms the FC agent with respect to final average score. A few problems appear here :
- The agent usually stays in the same square, not exploring a lot of the map.
- It also often chooses the move running into a poisonous cell, even if all the other moves would make it avoid the cell or visit a cell with cheese !

If we change the temperature to a low value ($0.3$ to $0.1$), we obtain slightly better performance on both CNN and FC agents. We can interpretate this as due to the fewer poisonous cells which outweight the fewer cells with cheese.
If we change the temperature to a high value ($0.3$ to $0.5$), we obtain significantly better performance on both CNN and FC agents. We can interpretate this as due to the higher probability to affect cheese to a cell, which automatically removes the possibility of assigning poison to it. We may note that the improvement in score being more significant than in the previous case could be explained by this priority of the chesse over the poison.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [108]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))
            agent.set_epsilon(agent.epsilon - (e+1)*(agent.epsilon-0.1) / epoch)

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train:
            reward -= self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward = reward + self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)
                               ),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


In [111]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.75, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train_explore, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/150 | Loss 0.0035 | Win/lose count 5.0/22.30000000000006 (-17.30000000000006)
Epoch 001/150 | Loss 0.0109 | Win/lose count 9.4/24.900000000000027 (-15.500000000000027)
Epoch 002/150 | Loss 0.0134 | Win/lose count 10.000000000000002/24.50000000000003 (-14.500000000000027)
Epoch 003/150 | Loss 0.0196 | Win/lose count 9.200000000000003/31.600000000000104 (-22.4000000000001)
Epoch 004/150 | Loss 0.0106 | Win/lose count 9.200000000000003/28.800000000000086 (-19.600000000000083)
Epoch 005/150 | Loss 0.0136 | Win/lose count 4.8/30.600000000000062 (-25.80000000000006)
Epoch 006/150 | Loss 0.0299 | Win/lose count 3.9999999999999996/31.50000000000011 (-27.50000000000011)
Epoch 007/150 | Loss 0.0211 | Win/lose count 4.8/30.900000000000144 (-26.100000000000144)
Epoch 008/150 | Loss 0.0208 | Win/lose count 10.100000000000005/32.400000000000105 (-22.3000000000001)
Epoch 009/150 | Loss 0.0323 | Win/lose count 2.4/30.50000000000012 (-28.100000000000122)
Epoch 010/150 | Loss 0.0496 | Win/lose

Epoch 085/150 | Loss 0.0095 | Win/lose count 1.2000000000000002/23.80000000000006 (-22.600000000000062)
Epoch 086/150 | Loss 0.0090 | Win/lose count 1.2000000000000002/22.800000000000047 (-21.600000000000048)
Epoch 087/150 | Loss 0.0189 | Win/lose count 0.8/22.90000000000005 (-22.100000000000048)
Epoch 088/150 | Loss 0.0086 | Win/lose count 1.2000000000000002/21.800000000000043 (-20.600000000000044)
Epoch 089/150 | Loss 0.0276 | Win/lose count 3.1999999999999997/21.300000000000022 (-18.100000000000023)
Epoch 090/150 | Loss 0.0095 | Win/lose count 1.6/23.70000000000006 (-22.10000000000006)
Epoch 091/150 | Loss 0.0181 | Win/lose count 1.2000000000000002/21.800000000000047 (-20.600000000000048)
Epoch 092/150 | Loss 0.0187 | Win/lose count 2.4/21.500000000000043 (-19.100000000000044)
Epoch 093/150 | Loss 0.0170 | Win/lose count 2.0/21.600000000000026 (-19.600000000000026)
Epoch 094/150 | Loss 0.0178 | Win/lose count 1.6/19.70000000000001 (-18.10000000000001)
Epoch 095/150 | Loss 0.0084 | W

In [115]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 4.0/3.0. Average score (1.0)
Win/lose count 1.0/1.0. Average score (0.5)
Win/lose count 1.0/4.0. Average score (-0.6666666666666666)
Win/lose count 2.0/2.0. Average score (-0.5)
Win/lose count 1.5/3.0. Average score (-0.7)
Win/lose count 2.5/2.0. Average score (-0.5)
Win/lose count 1.0/1.0. Average score (-0.42857142857142855)
Win/lose count 1.0/2.0. Average score (-0.5)
Win/lose count 2.0/3.0. Average score (-0.5555555555555556)
Win/lose count 1.5/3.0. Average score (-0.65)
Win/lose count 1.5/3.0. Average score (-0.7272727272727273)
Win/lose count 3.5/4.0. Average score (-0.7083333333333334)
Win/lose count 2.0/4.0. Average score (-0.8076923076923077)
Win/lose count 3.5/2.0. Average score (-0.6428571428571429)
Win/lose count 2.5/4.0. Average score (-0.7)
Win/lose count 2.5/2.0. Average score (-0.625)
Win/lose count 2.0/1.0. Average score (-0.5294117647058824)
Win/lose count 0.5/3.0. Average score (-0.6388888888888888)
Win/lose count 2.5/2.0. Average score (-0.57894736842

We denote here a slight improvement compared to the previous version of the CNN DQN Agent : our average score moves from the range $[-0.7, -0.9]$ to the range $[-0.55, -0.7]$. Moreover, we note that our agent is able to more explore the board.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***